# Sentiment Analysis
> * Positive or Negative

In [1]:
import os
from glob import glob
import pandas as pd
import html
import itertools
import pickle
import re

from collections import namedtuple
from gensim.models import doc2vec
import multiprocessing
from konlpy.tag import Mecab
from konlpy.utils import pprint
import numpy as np
from ckonlpy.tag import Twitter as ctwitter
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from gensim.models.doc2vec import TaggedDocument
mecab = Mecab()
ct = ctwitter()

## Term-existence

In [2]:
totalData = pd.read_csv('./data/sentiment_data/merged_rawdata_rating.txt',header=None, encoding='utf-8')
totalData.head()

,0,1
0,이젠 민주개혁의 길로 사설 이 90년대가 우리 모두에게 성취의 시대이기를 기원하고...,1
1,민주당과 공화당은 새해부터 정계개편을 적극 추진한다는 방침이며 양당의 김영삼ㆍ김종필...,1
2,"4당,지자제 연합공천 대비 파트너 탐색전 이같은 구상을 실현시키기 위해 민정당은...",1
3,90년대는 국내적으로 정치민주화와 선진국에로의 도약여부가 판가름날 것이고 한반도를 ...,1
4,"특히 노태우대통령이 3일 대국민 특별담화를 발표, 지난시대 의 완전청산을 선언한 후...",1


### mecab

In [3]:
def tokenize(doc):
    return ['/'.join(t) for t in mecab.pos(doc)]

In [4]:
docs = [(tokenize(totalData.loc[idx][0]), totalData.loc[idx][1]) for idx in totalData.index]

In [5]:
tokens = [t for d in docs for t in d[0]]
print(len(tokens))

72093332


In [6]:
import nltk
text = nltk.Text(tokens, name='rawdata')
print(text)

<Text: rawdata>


In [7]:
print(len(text.tokens))                 # returns number of tokens
print(len(set(text.tokens)))            # returns number of unique tokens
pprint(text.vocab().most_common(10))

72093332
303484
[('./SF', 2258735),
 ('을/JKO', 1596482),
 ('다/EF', 1552210),
 ('의/JKG', 1364800),
 ('이/JKS', 1240825),
 (',/SC', 1224528),
 ('에/JKB', 1201305),
 ('고/EC', 972908),
 ('를/JKO', 928322),
 ('는/ETM', 926895)]


In [8]:
selected_words = [f[0] for f in text.vocab().most_common(3000)]
def term_exists(doc):
    return {'exists({})'.format(word): (word in set(doc)) for word in selected_words}

In [9]:
train3, test3 = train_test_split(docs, test_size=0.2, random_state=42)

In [ ]:
train_xy = [(term_exists(d), c) for d, c in train3]
test_xy = [(term_exists(d), c) for d, c in test3]

In [13]:
classifier = nltk.NaiveBayesClassifier.train(train_xy)
print(nltk.classify.accuracy(classifier, test_xy))
# => 0.80418
classifier.show_most_informative_features(50)

0.624333333333
Most Informative Features
          exists(비난/NNG) = True                0 : 1      =    108.1 : 1.0
         exists(재배포/NNP) = True                0 : 1      =     81.0 : 1.0
          exists(무단/NNG) = True                0 : 1      =     80.8 : 1.0
          exists(전재/NNG) = True                0 : 1      =     68.4 : 1.0
          exists(금지/NNG) = True                0 : 1      =     52.6 : 1.0
          exists(최악/NNG) = True                0 : 1      =     44.7 : 1.0
            exists(ⓒ/SY) = True                0 : 1      =     43.0 : 1.0
   exists(heraldcorp/SL) = True                1 : 0      =     40.8 : 1.0
          exists(반발/NNG) = True                0 : 1      =     28.8 : 1.0
          exists(공세/NNG) = True                0 : 1      =     28.4 : 1.0
          exists(악재/NNG) = True                0 : 1      =     27.6 : 1.0
          exists(비리/NNG) = True                0 : 1      =     27.2 : 1.0
          exists(압박/NNG) = True                0 : 1      =

### twitter

In [16]:
def tokenize2(doc):
    return ['/'.join(t) for t in ct.pos(doc)]

In [20]:
docs2 = [(tokenize2(totalData.loc[idx][0]), totalData.loc[idx][1]) for idx in totalData.index]

KeyboardInterrupt: 

In [ ]:
tokens2 = [t for d in docs2 for t in d[0]]
print(len(tokens2))

In [ ]:
import nltk
text2 = nltk.Text(tokens2, name='rawdata2')
print(text2)

In [ ]:
print(len(text2.tokens))                 # returns number of tokens
print(len(set(text2.tokens)))            # returns number of unique tokens
pprint(text2.vocab().most_common(10))

In [ ]:
selected_words2 = [f[0] for f in text2.vocab().most_common(3000)]
def term_exists2(doc):
    return {'exists({})'.format(word): (word in set(doc2)) for word in selected_words2}

In [ ]:
train4, test4 = train_test_split(docs2, test_size=0.2, random_state=42)

In [ ]:
train_xy2 = [(term_exists2(d), c) for d, c in train4]
test_xy2 = [(term_exists2(d), c) for d, c in test4]

In [ ]:
classifier2 = nltk.NaiveBayesClassifier.train(train_xy2)
print(nltk.classify.accuracy(classifier2, test_xy2))
# => 0.80418
classifier2.show_most_informative_features(50)